## Import packages

In [8]:
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint
from CFModel import CFModel

In [2]:
CVSC_TRAIN_CSV_FILE = 'fb15k_cvsc_train.csv'
MODEL_WEIGHTS_FILE = 'fb15k_cvsc_weights.h5'
RNG_SEED = 1446557
K_FACTORS = 20

## Load CVSC training triples

In [3]:
triples = pd.read_csv(CVSC_TRAIN_CSV_FILE, sep='\t', usecols=['pid', 'rid']).sample(frac=1., random_state=RNG_SEED)

In [4]:
n_pairs = triples['pid'].max() + 1
m_relations = triples['rid'].max() + 1
print n_pairs, 'pairs,', m_relations, 'relations'

2995738 pairs, 26154 relations


## Define model

In [5]:
model = CFModel(n_pairs, m_relations, K_FACTORS)
model.compile(loss='mse', optimizer='adamax')

## Train model

In [ ]:
callbacks = [EarlyStopping('val_loss', patience=2), ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]

model.fit([triples['pid'].values, triples['rid'].values], np.ones(len(triples)), 
             nb_epoch=2, validation_split=.1, verbose=1, callbacks=callbacks)

Train on 6185264 samples, validate on 687252 samples
Epoch 1/2
  88128/6185264 [..............................] - ETA: 6644s - loss: 1.0000